<a href="https://colab.research.google.com/github/buon-viaggio/SOCAR-AI-BOOT-CAMP/blob/main/%ED%97%A4%EC%BB%A4%ED%86%A4_%EB%8B%A4%ED%95%A8%EA%BB%98%EC%B0%A8%EC%B0%BE%EC%9E%90_%EC%8F%98%EC%B9%B4%EC%A1%B4%ED%98%84%ED%99%A9%EC%88%98%EC%A7%91_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 쏘카존 거점 현황 가져오기

### IMPORT

In [ ]:
import pandas as pd
import requests
import time
#import schedule
from bs4 import BeautifulSoup as bs
from time import localtime, strftime
import datetime
from datetime import timedelta
import pytz

### 거점 정보 파일 불러오기

In [ ]:
# 파일 경로 입력 (디렉터리 까지만, 파일위치X)
garage = '경로'
apikey = 'API Key'
# 쏘카존 csv 파일명 입력 (socazone.csv)
socar = pd.read_csv(f'{garage}//socarzone.csv', encoding='UTF-8')
socarzoneId = socar['place_id'].values.tolist()

# 날짜 불러오기
utc = datetime.datetime.now(datetime.timezone.utc)
kst = utc + timedelta(hours=9)
date = kst.strftime('%Y-%m-%d')
time = kst.strftime('%H%M')
nowtime = kst.strftime('%Y-%m-%d %H:%M')

print(socarzoneId)

[2, 4, 18, 40, 47, 49, 51, 53, 55, 57, 61, 62, 64, 67, 77, 78, 80, 83, 90, 93, 102, 104, 105, 106, 107, 110, 113, 115, 119, 127, 128, 133, 134, 136, 138, 142, 143, 144, 146, 147, 152, 164, 169, 171, 174, 179, 181, 183, 184, 186, 191, 193, 195, 199, 200, 204, 205, 206, 207, 211, 228, 229, 242, 248, 249, 251, 254, 255, 263, 264, 269, 277, 281, 282, 284, 287, 291, 294, 300, 304, 305, 306, 310, 319, 321, 322, 323, 325, 327, 335, 337, 345, 347, 348, 352, 354, 360, 366, 367, 369, 370, 379, 386, 387, 396, 407, 411, 413, 416, 428, 435, 436, 444, 447, 448, 456, 457, 464, 468, 470, 473, 477, 482, 484, 485, 486, 489, 490, 494, 495, 498, 499, 506, 507, 513, 515, 519, 520, 522, 523, 529, 530, 535, 538, 540, 542, 545, 548, 554, 556, 560, 566, 567, 578, 580, 583, 596, 604, 606, 607, 608, 611, 616, 617, 626, 627, 628, 630, 637, 638, 643, 644, 646, 656, 659, 663, 664, 670, 671, 673, 680, 687, 691, 695, 699, 701, 705, 714, 721, 728, 729, 731, 734, 736, 737, 740, 741, 742, 748, 750, 752, 756, 759, 771, 7

### API 데이터 불러오기

In [ ]:
def get_Data():
    socarCnt = []
    utc = datetime.datetime.now(datetime.timezone.utc)
    kst = utc + timedelta(hours=9)
    nowtime = kst.strftime('%Y-%m-%d_%H%M')
    try:
        for id in socarzoneId:
            url = f'http://openapi.seoul.go.kr:8088/{apikey}/xml/NanumcarCarList/1/5/{id}/so'

            response = requests.get(url)
            if response.status_code == 200:
                try:
                    bs_content = bs(response.content, 'xml')
                    allcnt = bs_content.find('reservAbleCnt').text

                    socarCnt.append(allcnt)
                except Exception as e:
                    socarCnt.append(-100)
                    print(id, e)
                    continue
            else:
                socarCnt.append(-1)

    except Exception as e:
        outputdata = pd.DataFrame(socarCnt, columns=[f'{nowtime}'])
        # 에러시 저장되는 결과파일 (result_err.csv)
        outputdata.to_csv(f'{garage}//result_err-{nowtime}.csv')
        print(e)
        exit()

    outputdata = pd.DataFrame(socarCnt, columns=[f'{nowtime}'])
    # 결과파일 (result.csv)
    outputdata.to_csv(f'{garage}//result-{nowtime}.csv', mode='a')

In [ ]:
%%time
# 실행
get_Data()